# JuzData: proceso de generacion de causas - analisis temporal

En el presente notebook se analiza la columna 'exp_fechecho' de nuestro dataset, que contiene las fechas de los hechos que motivaron las causas judiciales. En consecuencia no vamos a meternos con el proceso de distribucion de causas, sino unicamente con su generacion; en particular, me interesa analizar si existe estacionalidad y autocorrelacion temporal en el proceso.

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import calmap
import warnings
warnings.filterwarnings('ignore')

In [2]:
df = pd.read_excel('penal_df_fec.xlsx')

FileNotFoundError: [Errno 2] No such file or directory: 'penal_df_fec.xlsx'

In [ ]:
df['exp_fechecho_dt'] = pd.to_datetime(df['exp_fechecho'])
df[['exp_fechecho','exp_fechecho_dt']].head(10)

In [ ]:
# convierto 'unknown' en NaN para contabilizar
import itertools as it
for i, col in it.product(df.index,df.columns):
    if df.at[i,col] == 'unknown':
        df.at[i,col] = np.nan

In [ ]:
df['exp_fechecho_dt'].notna().sum()

In [ ]:
calendar = df['exp_fechecho_dt'].value_counts()
calendar.head()

In [ ]:
years = [2015,2016,2017,2018,2019]
dias = ['Lun','Mar','Mie','Jue','Vie','Sab','Dom']
meses = ['Ene','Feb','Mar','Abr','May','Jun','Jul',
        'Ago','Sep','Oct','Nov','Dic']
for year in years:
    plt.figure(figsize=(15,10))
    calmap.yearplot(calendar, year=year,
                    daylabels=dias,
                   monthlabels=meses)
    plt.title(str(year) + ' - Causas por dia')
    file_name = 'calendar_heatmap_'+str(year)+'.png'
    plt.savefig(file_name)
    plt.show()

Que se puede ver en estos heatmaps: no parece haber una marcada estacionalidad (ver analisis mas abajo); el año 2016 presenta muchas mas causas que los demas (ver detalle abajo); y, lo que es mas importante a nuestros fines, se registra una tendencia a la autocorrelacion temporal de la cantidad de causas. Esto significa que los hechos que dan lugar a causas se agrupan en clusters temporales (como ya vimos que lo hacen geograficamente).

In [ ]:
# llevamos series a df
calendar_df = calendar.to_frame()
calendar_df = calendar_df.reset_index(drop=False)
calendar_df.head()

In [ ]:
year = pd.to_datetime('1/1/2015')
year

In [ ]:
# renombramos
calendar_df = calendar_df.rename(columns={'index':'date','exp_fechecho_dt':'causas'})
# reordenamos por fecha, reseteamos el indice
calendar_df = calendar_df.sort_values(by='date').reset_index(drop=True)
# ponemos un lag
for i in range(1,calendar_df.shape[0]):
    calendar_df.at[i,'causas-lag'] = calendar_df.at[i-1,'causas']
# filtramos desde 2015
year_filter = pd.to_datetime('1/1/2015')
calendar_df = calendar_df[calendar_df['date'] >= year_filter]
calendar_df.head()

In [ ]:
from scipy.stats import pearsonr, spearmanr, kendalltau

calendar_pearson = pearsonr(calendar_df['causas'],calendar_df['causas-lag'])
print(f'Autocorrelación Pearson: {calendar_pearson[0]}')
print('p-value Pearson: {:.20f}'.format(calendar_pearson[1]))
print()
calendar_spearman = spearmanr(calendar_df['causas'],calendar_df['causas-lag'])
print(f'Autocorrelación Spearman: {calendar_spearman[0]}')
print('p-value Spearman: {:.0f}'.format(calendar_spearman[1]))
print()
calendar_kendall = kendalltau(calendar_df['causas'],calendar_df['causas-lag'])
print(f'Autocorrelación Kendall: {calendar_kendall[0]}')
print('p-value Kendall: {:.20f}'.format(calendar_kendall[1]))
print()

Es decir, una autocorrelación grande, de signo positivo y significativa, para las tres medidas de correlación.

## Analisis por año

In [ ]:
df.columns

In [ ]:
df_year_groupby = df.groupby(df['exp_fechecho_dt'].dt.year)
df_year_count = df_year_groupby.count()['exp_cuij'].to_frame(name='causas')
df_year_count

In [ ]:
anios = ['2015','2016','2017','2018','2019']
plt.figure(figsize=(10,7))
plt.bar(x=anios,
        height=df_year_count.values.flatten()[-5:])
year_avg = np.mean(df_year_count.values.flatten()[-5:])
plt.plot([year_avg for i in range(5)],c='red')
plt.annotate('Promedio: '+str(int(year_avg)),
             (3,2700),c='red',fontsize=13,
            fontweight='bold')
plt.title('Causas por Año')
plt.savefig('causas_por_anio.png')
plt.show()

Estas diferencias entre la cantidad de causas por año muy probablemente se deba a complicaciones en la migración de datos entre bases (como hablamos en la última reunión con Yasmin Quiroga).

In [ ]:
df_year_count.index = df_year_count.index.astype(int)
df_year_count.index = df_year_count.index.astype(str)

In [ ]:
calendar_level = pd.DataFrame()
for i,index in enumerate(calendar.index):
    calendar_level.at[i,'fecha'] = index
    calendar_level.at[i,'causas'] = calendar[index]
    # llevamos el año al type de df_year_count
    year = str(index.year)
    causas_year = df_year_count.at[year,'causas']
    calendar_level.at[i,'causas-level'] = calendar[index] / causas_year

In [ ]:
calendar_level.head()

In [ ]:
calendar_level['fecha'] = pd.to_datetime(calendar_level['fecha'])

In [ ]:
calendar_level_series = pd.Series(data=calendar_level['causas-level'].values,
                                  index=calendar_level['fecha'])

In [ ]:
calendar_level_series.head()

## Autocorrelación por Año

In [ ]:
def print_autocorr(year):
    df = calendar_df[calendar_df['date'].dt.year == year]
    print(f'Año: {year}')
    print()
    calendar_pearson = pearsonr(df['causas'],df['causas-lag'])
    print(f'Autocorrelación Pearson: {calendar_pearson[0]}')
    print('p-value Pearson: {:.20f}'.format(calendar_pearson[1]))
    print()
    calendar_spearman = spearmanr(df['causas'],df['causas-lag'])
    print(f'Autocorrelación Spearman: {calendar_spearman[0]}')
    print('p-value Spearman: {:.20f}'.format(calendar_spearman[1]))
    print()
    calendar_kendall = kendalltau(df['causas'],df['causas-lag'])
    print(f'Autocorrelación Kendall: {calendar_kendall[0]}')
    print('p-value Kendall: {:.20f}'.format(calendar_kendall[1]))
    print()

In [ ]:
pearsonr(veamos['causas'],veamos['causas-lag'])[0]

In [ ]:
veamos = calendar_df[calendar_df['date'].dt.year == 2016]

In [ ]:
for year in [2015,2016,2017,2018,2019]:
    print_autocorr(year)

## Autocorrelación: Dickey-Fuller, Ploteo

In [ ]:
import statsmodels.formula.api as smf  
import statsmodels.tsa.api as smt
import statsmodels.api as sm
import scipy.stats as scs

In [ ]:
def tsplot(y, lags=None, figsize=(12, 7), style='bmh', title='Title'):
    """
        Plot time series, its ACF and PACF, calculate Dickey–Fuller test
        
        y - timeseries
        lags - how many lags to include in ACF, PACF calculation
    """
    if not isinstance(y, pd.Series):
        y = pd.Series(y)
        
    with plt.style.context(style):    
        fig = plt.figure(figsize=figsize)
        layout = (2, 2)
        ts_ax = plt.subplot2grid(layout, (0, 0), colspan=2)
        acf_ax = plt.subplot2grid(layout, (1, 0))
        pacf_ax = plt.subplot2grid(layout, (1, 1))
        
        y.plot(ax=ts_ax)
        p_value = sm.tsa.stattools.adfuller(y)[1]
        ts_ax.set_title(title+'\n Dickey-Fuller: p={0:.5f}'.format(p_value))
        smt.graphics.plot_acf(y, lags=lags, ax=acf_ax)
        smt.graphics.plot_pacf(y, lags=lags, ax=pacf_ax)
        plt.tight_layout()

In [ ]:
for year in [2015,2016,2017,2018,2019]:
    series = calendar_df[calendar_df['date'].dt.year == year]['causas']
    tsplot(series, lags=10, title='Time Series Año '+str(year))

## Estacionalidad: causas por meses

In [ ]:
df_month_groupby = df.groupby(df['exp_fechecho_dt'].dt.month)
df_month_count = df_month_groupby.count()['exp_cuij'].to_frame(name='causas')
df_month_count

In [ ]:
month_avg = np.mean(df_month_count.values)
month_avg

In [ ]:
meses = ['Ene','Feb','Mar','Abr','May','Jun',
         'Jul','Ago','Sep','Oct','Nov','Dic']
plt.figure(figsize=(15,6))
plt.bar(x=meses,
        height=df_month_count.values.flatten())
plt.plot([month_avg for i in range(12)],c='red')
plt.annotate('Promedio: '+str(int(month_avg)),
             (9,1120),c='red',fontsize=13,
            fontweight='bold')
plt.title('Causas por Mes')
plt.savefig('causas_por_mes.png')
plt.show()

No parece haber fuerte estacionalidad en los meses (faltaría un test formal).

## Estacionalidad: causas por dias

In [ ]:
df_weekday_groupby = df.groupby(df['exp_fechecho_dt'].dt.weekday)
df_weekday_count = df_weekday_groupby.count()['exp_cuij'].to_frame(name='causas')
df_weekday_count

In [ ]:
weekday_avg = np.mean(df_weekday_count.values)
weekday_avg

In [ ]:
dias = ['Lun','Mar','Mie','Jue','Vie','Sab','Dom']
plt.figure(figsize=(15,6))
plt.bar(x=dias,
        height=df_weekday_count.values.flatten())
plt.plot([weekday_avg for i in range(7)],c='red')
plt.annotate('Promedio: '+str(int(weekday_avg)),
             (5,1900),c='red',fontsize=13,
            fontweight='bold')
plt.title('Causas por Mes')
plt.savefig('causas_por_mes.png')
plt.show()

No parece haber fuerte estacionalidad en los días de la semana.

## Discusión Conceptual

- Autocorrelación temporal: modelos empíricos, componente teórico débil (inercia). No aplicaría en nuestro caso. El origen puede estar en la captación de datos? (lo hablamos en la última call, lo sugiere Martina en la minuta).
- Autocorrelación espacial: muy utilizada en criminología. Problema econométrico de selection bias. Teoría relacionada: broken window (no confundir con Rudolph Giuliani).
- Autocorrelación temporal en criminología: efecto contagio (cómo operaría?)

## Expedientes

In [ ]:
# separo exp_cuij en tres columnas
for i in df.index:
    df.at[i,'exp_cuij_1'] = df.at[i,'exp_cuij'][0:2]
    df.at[i,'exp_cuij_2'] = df.at[i,'exp_cuij'][3:11]
    df.at[i,'exp_cuij_3'] = df.at[i,'exp_cuij'][12:13]

In [ ]:
df.shape

In [ ]:
# expedientes unicos segun exp_cuij
df['exp_cuij_2'].unique().shape

In [ ]:
# expedientes unicos segun exp_id
df['exp_id'].unique().shape